In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

import bezier

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

from scipy import spatial, stats
from scipy.ndimage import gaussian_filter
from scipy.integrate import odeint
from functools import partial

import fn
from genpen.axicam import AxiCam

In [ ]:
# make page
paper_size = '6x6 inches'
border:float=10
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
center = drawbox.centroid

In [ ]:
def quadrant_split(square_poly):
    sq = gp.Poly(square_poly)
    upper_left = box(minx=sq.left, miny=sq.p.centroid.y, maxx=sq.p.centroid.x, maxy=sq.top, ccw=True)
    upper_right = box(minx=sq.p.centroid.x, miny=sq.p.centroid.y, maxx=sq.right, maxy=sq.top, ccw=True)
    lower_right = box(minx=sq.p.centroid.x, miny=sq.bottom, maxx=sq.right, maxy=sq.p.centroid.y, ccw=True)
    lower_left = box(minx=sq.left, miny=sq.bottom, maxx=sq.p.centroid.x, maxy=sq.p.centroid.y, ccw=True)
    quadrants = gp.merge_Polygons([upper_left, upper_right, lower_right, lower_left])
    return quadrants

def permit_by_index(i, p, allowed_indices):
    return i in allowed_indices

def bino_draw(i, p, p_continue=1):
    return np.random.binomial(n=1, p=p_continue)


def permit_all(i, p):
    return True

def flex_rule_recursive_split(poly, split_func, continue_func, depth=0, depth_limit=15, buffer_kwargs=None):
    
    if buffer_kwargs is None:
        buffer_kwargs = {'distance':0}
    polys = split_func(poly)
    split_polys = []
    for i, p in enumerate(polys):
        continue_draw = continue_func(i=i, p=p)
        
        if continue_draw and (depth<depth_limit):
            
            split_polys += flex_rule_recursive_split(
                p, split_func=split_func, continue_func=continue_func, 
                depth=depth+1, depth_limit=depth_limit,
                buffer_kwargs=buffer_kwargs
            ) 
        else:
            split_polys.append(p.buffer(**buffer_kwargs))
    return split_polys

In [ ]:
upper_left_and_lower_right = partial(permit_by_index, allowed_indices=[0, 2])

In [ ]:
all_but_lower_left = partial(permit_by_index, allowed_indices=[0,1, 2])

In [ ]:
high_p = partial(bino_draw, p_continue=0.9)
low_p = partial(bino_draw, p_continue=0.7)

In [ ]:
polys = flex_rule_recursive_split(poly=drawbox, split_func=quadrant_split, continue_func=permit_all, depth_limit=2, buffer_kwargs={'distance':1e-6})

In [ ]:
black_squares = gp.merge_Polygons(polys[::2])

In [ ]:
white_squares = gp.merge_Polygons(polys[1::2])

In [ ]:
top_bs_subs = []
for bs in black_squares:
    top_bs_subs += flex_rule_recursive_split(poly=bs, split_func=quadrant_split, 
                                             continue_func=permit_all, 
                                             depth_limit=0)

In [ ]:

bs_subs = []
for bs in top_bs_subs:
    bs_subs += flex_rule_recursive_split(poly=bs, split_func=quadrant_split, 
                                             continue_func=high_p, 
                                             depth_limit=3)
    
    
bs_subs = gp.merge_Polygons([bs_subs, top_bs_subs])

In [ ]:
size_order = np.argsort([-b.area for b in bs_subs])
bs_subs = gp.merge_Polygons([bs_subs[ii] for ii in size_order])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

sk.geometry(white_squares.boundary)
sk.geometry(black_squares.boundary)
sk.geometry(bs_subs.boundary)

sk.display(color_mode='layer')

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

counter = 0
for ls in white_squares.boundary:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    
for ls in black_squares.boundary:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    
for ls in bs_subs.boundary:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    

sk.display(color_mode='layer')

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
ac = AxiCam()

In [ ]:
ac.init_cam()

In [ ]:
ac.cam.video_preview()

In [ ]:
ac.cam.close()

In [ ]:
ac.ad.options.speed_penup = 80

In [ ]:
ac.ad.options.accel = 70
ac.ad.options.pen_raise_rate = 85

In [ ]:
ac.ad.options.speed_pendown = 10
ac.ad.update()

In [ ]:
ac.plot_layers(wait_times=0.3, start_layer=0)

# more itnerestin

In [ ]:
class ContinuePolicy(object):
    
    def __init__(self, policy):
        self.policy = policy
        
    def __call__(self, i, p, depth, poly):
        return self.policy(i, p, depth, poly)

def even_odd(i, p, depth, poly):
    lookup = {
        0: [0, 2],
        1: [1, 3]
    }
    mod = depth % 2
    return i in lookup.get(mod)

def mod4(i, p, depth, poly):
    lookup = {
        0: [0],
        1: [1],
        2: [2],
        3: [3]
    }
    mod = depth % 4
    return i in lookup.get(mod)

def permit_by_index_depth_dependent(i, p, allowed_index_lookup):
    return 

def very_flex_rule_recursive_split(poly, split_func, continue_func, depth=0, depth_limit=15, buffer_kwargs=None):
    
    if buffer_kwargs is None:
        buffer_kwargs = {'distance':0}
    polys = split_func(poly)
    split_polys = []
    for i, p in enumerate(polys):
        continue_draw = continue_func(i, p, depth, poly)
        
        if continue_draw and (depth<depth_limit):
            
            split_polys += very_flex_rule_recursive_split(
                p, split_func=split_func, continue_func=continue_func, 
                depth=depth+1, depth_limit=depth_limit,
                buffer_kwargs=buffer_kwargs
            ) 
        else:
            split_polys.append(p.buffer(**buffer_kwargs))
    return split_polys

def distance_from_pt(i, p, depth, poly, target, p_range, d_range):
    d = poly.distance(target)
    p_continue = np.interp(d, d_range, p_range)
    return np.random.binomial(n=1, p=p_continue)

In [ ]:
upper_left_and_lower_right = partial(permit_by_index, allowed_indices=[0, 2])

In [ ]:
all_but_lower_left = partial(permit_by_index, allowed_indices=[0,1, 2])

In [ ]:
high_p = partial(bino_draw, p_continue=0.9)
low_p = partial(bino_draw, p_continue=0.7)

In [ ]:
polys = flex_rule_recursive_split(poly=drawbox, split_func=quadrant_split, continue_func=permit_all, depth_limit=2, buffer_kwargs={'distance':1e-6})

black_squares = gp.merge_Polygons(polys[::2])

white_squares = gp.merge_Polygons(polys[1::2])

dist_from_center = partial(distance_from_pt, target=drawbox.centroid, p_range=(1., 0.35,), d_range=(35, 75))

cp = ContinuePolicy(dist_from_center)

top_bs_subs = []
# for bs in black_squares:
#     top_bs_subs += very_flex_rule_recursive_split(poly=bs, split_func=quadrant_split, 
#                                              continue_func=cp, 
#                                              depth_limit=0)


bs_subs = []
for bs in black_squares:
    bs_subs += very_flex_rule_recursive_split(poly=bs, split_func=quadrant_split, 
                                             continue_func=cp, 
                                             depth_limit=4)
    
    
bs_subs = gp.merge_Polygons([bs_subs, top_bs_subs])

In [ ]:
size_order = np.argsort([-b.area for b in bs_subs])
bs_subs = gp.merge_Polygons([bs_subs[ii] for ii in size_order])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

# sk.geometry(white_squares.boundary)
sk.geometry(black_squares.boundary)
sk.geometry(bs_subs.boundary)

sk.display(color_mode='layer')

In [ ]:
import fn

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
ac = AxiCam()

In [ ]:
ac.init_cam()

In [ ]:
ac.cam.video_preview()

In [ ]:
ac.cam.get_video(5)

# more itnerestin

In [ ]:
class ContinuePolicy(object):
    
    def __init__(self, policy):
        self.policy = policy
        
    def __call__(self, i, p, depth, poly):
        return self.policy(i, p, depth, poly)

def even_odd(i, p, depth, poly):
    lookup = {
        0: [0, 2],
        1: [1, 3]
    }
    mod = depth % 2
    return i in lookup.get(mod)

def mod4(i, p, depth, poly):
    lookup = {
        0: [0],
        1: [1],
        2: [2],
        3: [3]
    }
    mod = depth % 4
    return i in lookup.get(mod)

def permit_by_index_depth_dependent(i, p, allowed_index_lookup):
    return 

def very_flex_rule_recursive_split(poly, split_func, continue_func, depth=0, depth_limit=15, buffer_kwargs=None):
    
    if buffer_kwargs is None:
        buffer_kwargs = {'distance':0}
    polys = split_func(poly)
    split_polys = []
    for i, p in enumerate(polys):
        continue_draw = continue_func(i, p, depth, poly)
        
        if continue_draw and (depth<depth_limit):
            
            split_polys += very_flex_rule_recursive_split(
                p, split_func=split_func, continue_func=continue_func, 
                depth=depth+1, depth_limit=depth_limit,
                buffer_kwargs=buffer_kwargs
            ) 
        else:
            split_polys.append(p.buffer(**buffer_kwargs))
    return split_polys

def distance_from_pt(i, p, depth, poly, target, p_range, d_range):
    d = poly.distance(target)
    p_continue = np.interp(d, d_range, p_range)
    return np.random.binomial(n=1, p=p_continue)

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=30
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
center = drawbox.centroid

In [ ]:
drawbox.bounds

In [ ]:
polys = flex_rule_recursive_split(poly=drawbox, split_func=quadrant_split, continue_func=permit_all, depth_limit=3, buffer_kwargs={'distance':1e-6})

black_squares = gp.merge_Polygons(polys[::2])

white_squares = gp.merge_Polygons(polys[1::2])

dist_from_center = partial(distance_from_pt, target=drawbox.centroid, p_range=(0.95, 0.1,), d_range=(15, 130))

cp = ContinuePolicy(dist_from_center)

top_bs_subs = []
# for bs in black_squares:
#     top_bs_subs += very_flex_rule_recursive_split(poly=bs, split_func=quadrant_split, 
#                                              continue_func=cp, 
#                                              depth_limit=0)


bs_subs = []
for bs in black_squares:
    bs_subs += very_flex_rule_recursive_split(poly=bs, split_func=quadrant_split, 
                                             continue_func=cp, 
                                             depth_limit=2)
    


# dist_from_center = partial(distance_from_pt, target=drawbox.centroid, p_range=(0.1, 0.95), d_range=(15, 130))
# cp = ContinuePolicy(dist_from_center)
# ws_subs = []
# for ws in white_squares:
#     ws_subs += very_flex_rule_recursive_split(poly=ws, split_func=quadrant_split, 
#                                              continue_func=cp, 
#                                              depth_limit=3)
    
    
bs_subs = gp.merge_Polygons([bs_subs, top_bs_subs])
ws_subs = gp.merge_Polygons([ws_subs])

In [ ]:
coords = []
for bs in bs_subs:
    coords += [Point(c) for c in bs.boundary.coords]

In [ ]:
cps = MultiPoint(coords)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

sk.geometry
# sk.geometry(white_squares.boundary)
# sk.geometry(black_squares.boundary)
# sk.geometry(bs_subs.boundary)
# sk.geometry(ws_subs.boundary)
sk.geometry(gp.merge_LineStrings(so.triangulate(cps, edges=True)))

sk.display(color_mode='layer')

In [ ]:
import fn

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)